In [1]:
import pandas as pd
import numpy as np
 
off_train= pd.read_csv("data/ccf_offline_stage1_train.csv",header=0) # 跳过header
online_train = pd.read_csv("data/ccf_online_stage1_train.csv",header=0)
off_predict =pd.read_csv("data/ccf_offline_stage1_test_revised.csv",header=0)
off_train.columns = ["user_id","merchant_id","coupon_id","discount_rate","distance","date_received","date"]
online_train.columns = ["user_id","merchant_id","action","coupon_id","discount_rate","date_received","date"]
off_predict.columns = ["user_id","merchant_id","coupon_id","discount_rate","distance","date_received"]

In [2]:
print off_train[((off_train.date_received=='null') & (off_train.date=='null'))].shape
print online_train[((online_train.date_received=='null') & (online_train.date=='null'))].shape
print "off_train online_train 没有date_received 和 data 同时为NULL 的情况"

print "off_train记录数目："+str(off_train.shape[0])
print "off_train领取优惠券数量："+str(off_train[(off_train.date_received!='null')].shape[0])
print "off_train优惠券种类： "+str(off_train[off_train.coupon_id!='null'].coupon_id.value_counts().shape[0])
print off_train.drop_duplicates(["merchant_id","coupon_id"]).shape

# def f(s):
#     return pd.Series({ 'n':s["merchant_id"].drop_duplicates().shape[0]})
# off_train.groupby(["coupon_id"]).apply(f)
print "一个coupon_id 可能对应多个merchat_id"


print "off_train用户个数："+str(off_train.user_id.value_counts().shape[0])
print "date_received区间："+ str(off_train[off_train["date_received"]!="null"].date_received.min()) +" 到 "+str(off_train[off_train["date_received"]!="null"].date_received.max())
print "date 区间：" +str(off_train[off_train["date"]!="null"].date.min())+" 到 "+ str(off_train[off_train["date"]!="null"].date.max())



(0, 7)
(0, 7)
off_train online_train 没有date_received 和 data 同时为NULL 的情况
off_train记录数目：1754884
off_train领取优惠券数量：1053282
off_train优惠券种类： 9738
(17888, 7)
一个coupon_id 可能对应多个merchat_id
off_train用户个数：539438
date_received区间：20160101 到 20160615
date 区间：20160101 到 20160630


In [3]:
off_train.discount_rate.value_counts()["null"]

701602

In [4]:
off_train_user_id = off_train.user_id.drop_duplicates()
off_predict_user_id = off_predict.user_id.drop_duplicates()
online_train_user_id = online_train.user_id.drop_duplicates()
off_train_user_id.index = off_train_user_id.values
off_predict_user_id.index = off_predict_user_id.values
online_train_user_id.index = online_train_user_id.values

print "off_predict 有"+str (off_predict_user_id.size)+"用户"
print "off_predict 有"+str(off_predict_user_id.index.intersection(online_train_user_id.index).shape[0])+"用户 与 online_train用户有交集"
print "off_predict 有"+str(off_predict_user_id.index.intersection(off_train_user_id.index).shape[0])+"用户 与 off_train用户有交集"
print 

off_train_merchant_id  = off_train.merchant_id.drop_duplicates()
off_predict_merchant_id = off_predict.merchant_id.drop_duplicates()
online_train_merchant_id = online_train.merchant_id.drop_duplicates()
off_train_merchant_id.index = off_train_merchant_id.values
off_predict_merchant_id.index = off_predict_merchant_id.values
online_train_merchant_id.index = online_train_merchant_id.values

print  "off_predict有"+str(off_predict_merchant_id.size)+"商家"
print  "off_predict 有"+str(off_predict_merchant_id.index.intersection(online_train_merchant_id.index).shape[0])+"商家 与 online_train 商家有交集"
print  "off_predict 有"+str(off_predict_merchant_id.index.intersection(off_train_merchant_id.index).shape[0])+"商家 与 off_train 商家有交集"
print

off_predict_coupon_id = off_predict.coupon_id.drop_duplicates()
off_train_coupon_id = off_train.coupon_id.drop_duplicates()
off_predict_coupon_id.index = off_predict_coupon_id.values
off_train_coupon_id.index = off_train_coupon_id.values

print "off_predict有"+ str(off_predict_coupon_id.size)+"优惠券"
print  "off_predict有"+str(off_predict_coupon_id.index.intersection(off_train_merchant_id.index).shape[0])+"优惠券　与 off_train 优惠券有交集"

off_predict 有76309用户
off_predict 有43155用户 与 online_train用户有交集
off_predict 有76307用户 与 off_train用户有交集

off_predict有1559商家
off_predict 有0商家 与 online_train 商家有交集
off_predict 有1558商家 与 off_train 商家有交集

off_predict有2050优惠券
off_predict有1269优惠券　与 off_train 优惠券有交集


In [36]:
#划分数据集为训练集和测试集
# 特征窗: feature 和 标签窗: dataset
dataset1 = off_train[((off_train.date_received>="20160414")&(off_train.date_received<="20160514"))]
feature1 = off_train[(((off_train.date>="20160101" ) & (off_train.date<="20160414")) | ((off_train.date=="null") &(off_train.date_received>="20160101")&(off_train.date_received<="20160414")))]

dataset2 = off_train[(off_train.date_received>="20160514") & (off_train.date_received<="20160615")]
feature2 = off_train[((off_train.date>="20160201")  & (off_train.date<="20160514")) |( (off_train.date=="null")  & (off_train.date_received>="20160201") &(off_train.date_received<="20160514"))]

dataset3= off_predict
feature3 = off_train[((off_train.date>="20160315")  & (off_train.date<="20160701")) |( (off_train.date=="null")  & (off_train.date_received>="20160315") &(off_train.date_received<="20160701"))]


## other feature

In [77]:
#for dataset3
from datetime import date
t = dataset3[["user_id"]]
t["this_month_user_receive_all_coupon"]=1
t=t.groupby(["user_id"]).agg("sum").reset_index()
 

t1 = dataset3[["user_id","coupon_id"]]
t1["this_month_user_receive_same_coupon"]=1
t1= t1.groupby(["user_id","coupon_id"]).agg("sum").reset_index()
 

t2= dataset3[["user_id","coupon_id","date_received"]]
t2.date_received = t2.date_received.astype('str') # 必须得astype(str)转化为string,默认是object 类型，后面要使用join 对于字符串连接
t2  =t2[(t2.date_received !="null")]
t2 = t2.groupby(['user_id','coupon_id']).agg(lambda x:':'.join(x)).reset_index()
def f(s):
    return len(s.split(":"))
t2["receive_number"] = t2.date_received.apply(f)
t2= t2[(t2.receive_number>1)]

    
t2["max_date_received"]=t2.date_received.apply(lambda x: max([ int(d) for d in x.split(":")])  )
t2["min_date_received"]=t2.date_received.apply(lambda x:min([int(d) for d in x.split(":")]) )
t2=t2[["user_id","coupon_id","max_date_received","min_date_received"]]

t3= dataset3[["user_id","coupon_id","date_received"]]
t3=pd.merge(t3,t2,on=["user_id","coupon_id"],how="left")
t3["this_month_user_receive_same_coupon_lastone"]=t3["max_date_received"]-t3["date_received"]
t3["this_month_user_receive_same_coupon_firstone"]=t3["date_received"]-t3["min_date_received"]

def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1  # 仅仅收到一次
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)


t4=dataset3[["user_id","date_received"]]
t4["this_day_user_receive_all_coupon_count"]=1
t4= t4.groupby(["user_id","date_received"]).agg("sum").reset_index()

t5 = dataset3[["user_id","coupon_id","date_received"]]
t5["this_day_user_receive_same_coupon_count"]=1
t5=t5.groupby(["user_id","coupon_id","date_received"]).agg("sum").reset_index()

t6= dataset3[["user_id","coupon_id","date_received"]]
t6.date_received = t6.date_received.astype('str')
t6=t6.groupby(["user_id","coupon_id"]).agg(lambda x : ":".join(x)).reset_index()
t6.rename(columns={"date_received":"dates"},inplace=True)


t7= dataset3[["user_id","coupon_id","date_received"]]
t7= pd.merge(t7,t6, on=["user_id","coupon_id"],how="left")
t7= t7.head(10)
def day_gap_before(s):
    date_received = s["date_received"]
    dates = s["dates"].split(":")
    print date_received
    gaps=[]
    # date 类型可以加减操作
    for d in dates:
       
        gap =( date(int(date_received[0:4]),int(date_received[4:6]), int (date_received[6:8])) - date(int(d[0:4]),int(d[4:6]), int (d[6:8]))     ).days  
        if gap>0:
            gaps.append(gap)
    if (len(gaps)==0):
        return -1
    return min(gaps)

t7.dates=t7.dates.astype("str")
t7.date_received=t7.date_received.astype("str")
t7["day_gap_before"]= t7[["dates","date_received"]].apply(day_gap_before,axis = 1)
print t7.head(10)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

20160712
20160712


TypeError: ("can't compare datetime.timedelta to int", u'occurred at index 0')

## coupon feature
优惠比率:discount_rate ; 历史领取次数：receive_count; 销核次数：use_count; 销核比率：use_rate; 领取日期是周几：day_of_week; 领取日期是一个月第几天：day_of_month;

In [10]:
# dataset1
# 优惠比率特征
def cal_discount_rate(s):
    s=str(s)
    if s =="null":
        return np.NaN
    else:
        splited = s.split(":")
        if len(splited)==1:
            return float(splited[0])
        else:
            return 1- float(splited[1])/float(splited[0])
dataset1["discount_rate"] = dataset1.discount_rate.apply(cal_discount_rate)

# # 历史领取次数
# d =dataset1[["coupon_id"]]
# d["receive_count"]=1
# d2 = d.groupby(["coupon_id"]).agg("sum").reset_index()
# dataset1 = pd.merge(dataset1,d2,on="coupon_id",how="left")

# # 销核次数
# d=dataset1[["coupon_id","date","date_received"]]
# d2 =d[(d.date!="null"  )& (d.date_received!="null")]
# d2["use_count"]=1
# d3= d2.groupby(["coupon_id"]).agg("sum").reset_index()
# dataset1 = pd.merge(dataset1,d2,on="coupon_id",how="left")

# dataset1.use_rate = dataset1.use_count/dataset1.receive_count

dataset1['day_of_week'] = dataset1.date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])))
print dataset1.head(10)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


ValueError: invalid literal for int() with base 10: 'null'

In [54]:
dataset3.user_id.isin([448]).value_counts()

False    113635
True          5
Name: user_id, dtype: int64